# Подготовка модели распознавания рукописных букв и цифр

In [2]:
!pip install torch torchvision torchinfo
!pip install torch torchvision

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import EMNIST
from model import ImprovedCNNModel

# Определение устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Мягкие трансформации для Data Augmentation
transform = transforms.Compose([
    transforms.RandomRotation(5),           
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Загрузка и подготовка данных
train_dataset = EMNIST(root='data', split='balanced', train=True, download=True, transform=transform)
test_dataset = EMNIST(root='data', split='balanced', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Создание экземпляра модели и отправка на устройство
model = ImprovedCNNModel().to(device)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Обучение модели
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Оценка модели на тестовых данных
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

# Сохранение модели
torch.save(model.state_dict(), 'myapp/model.ckpt')


/home/dmitriy/anaconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Epoch [1/10], Loss: 0.8954
Epoch [2/10], Loss: 0.4761
Epoch [3/10], Loss: 0.4249
Epoch [4/10], Loss: 0.3934
Epoch [5/10], Loss: 0.3738
Epoch [6/10], Loss: 0.3590
Epoch [7/10], Loss: 0.3451
Epoch [8/10], Loss: 0.3325
Epoch [9/10], Loss: 0.3228
Epoch [10/10], Loss: 0.3122
Accuracy of the model on the test images: 88.67%
